In [4]:
import os
from tensorflow.keras.models import load_model
import tf2onnx
import onnx
model = load_model('/data/wang_sc/HAR-With-SmartWatch/attention_based/LIMUNet_trained.h5')
onnx_model, _ = tf2onnx.convert.from_keras(model)
onnx.save(onnx_model, 'model.onnx')


2024-01-14 20:21:01.867891: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 4
2024-01-14 20:21:01.878573: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18231 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6
2024-01-14 20:21:01.879166: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 18235 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:5b:00.0, compute capability: 8.6
2024-01-14 20:21:01.879996: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22288 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:9b:00.0, compute capability: 8.6
2024-01-14 20:21:01.880406: I tensorflow/core/common_runtime/gpu/gpu_

In [5]:
check = onnx.checker.check_model(onnx_model)
print('Check: ', check)

Check:  None


In [6]:
import tensorflow as tf
import onnxruntime
import numpy as np

input1 = np.random.random((1,171,9,1)).astype('float32')

ort_sess = onnxruntime.InferenceSession("./model.onnx")
ort_inputs = {ort_sess.get_inputs()[0].name: input1}
ort_outs = ort_sess.run(None, ort_inputs)

tf_model = tf.keras.models.load_model("/data/wang_sc/HAR-With-SmartWatch/attention_based/LIMUNet_trainedProcessed_self_made.h5")
tf_outs = tf_model(inputs=input1)

print(ort_outs[0])
print(tf_outs.numpy())
np.testing.assert_allclose(tf_outs.numpy(), ort_outs[0], rtol=1e-03, atol=1e-05)
print("onnx model check finsh.")


[[5.2074502e-05 1.9688366e-02 1.3663396e-06 9.6709911e-05 5.9941221e-02
  3.3219883e-05 2.7692570e-06 2.9943665e-06 7.2272815e-04 3.0542447e-05
  1.4822512e-06 9.0249572e-03 9.1040164e-01]]
[[1.6350237e-09 4.8682551e-08 1.3102162e-07 5.0843628e-08 3.4547892e-07
  1.9859088e-10 7.4177073e-09 2.8393798e-11 6.2881654e-08 3.6819611e-04
  1.9214822e-04 9.4178551e-01 5.7653621e-02]]


AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

Mismatched elements: 10 / 13 (76.9%)
Max absolute difference: 0.93276054
Max relative difference: 128.63269
 x: array([[1.635024e-09, 4.868255e-08, 1.310216e-07, 5.084363e-08,
        3.454789e-07, 1.985909e-10, 7.417707e-09, 2.839380e-11,
        6.288165e-08, 3.681961e-04, 1.921482e-04, 9.417855e-01,
        5.765362e-02]], dtype=float32)
 y: array([[5.207450e-05, 1.968837e-02, 1.366340e-06, 9.670991e-05,
        5.994122e-02, 3.321988e-05, 2.769257e-06, 2.994366e-06,
        7.227281e-04, 3.054245e-05, 1.482251e-06, 9.024957e-03,
        9.104016e-01]], dtype=float32)